In [1]:
import sys
sys.path.append('../../../')

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

from keras.layers import Dense, Dropout

import tensorflow as tf
import numpy as np
from tqdm import tqdm
from keras.layers import Dense, Dropout, GRU

import pandas as pd
import tensorflow as tf
import numpy as np

from machine_learning.models import FeedzaiConcatProduction
from data_processing.batch_generator import load_test_set
tf.random.set_seed(42)

define model:

In [3]:
model = FeedzaiConcatProduction()


In [4]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
        optimizer=tf.keras.optimizers.Adam(),
        metrics=[
            tf.keras.metrics.BinaryAccuracy(),
            tf.keras.metrics.TruePositives(),
            tf.keras.metrics.TrueNegatives(),
            tf.keras.metrics.FalsePositives(),
            tf.keras.metrics.FalseNegatives(),
            tf.keras.metrics.Precision(),
            tf.keras.metrics.Recall(),
            tf.keras.metrics.AUC()])


### Dataset

In [5]:
path = '../../../'

In [6]:
sample_transaction = np.load(f'{path}data/test/all_transactions.npy')[0]
test_labels = np.load(f'{path}data/test/all_labels.npy').astype(float)
test_set = tf.data.Dataset.from_tensor_slices(
    (np.load(f'{path}data/test/all_transactions.npy'), 
     test_labels)
).batch(1)
# initialize weights
model(np.expand_dims(sample_transaction, axis=0))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[1.5121839e-23]], dtype=float32)>

### Evaluate Model

In [7]:
train_df = pd.read_csv(f'{path}training_runs/train_{model.name}.csv')
train_df['f1_score'] = 2 * (train_df['precision'] * train_df['recall']) / (train_df['precision'] + train_df['recall'])
train_df

,epoch,loss,binary_accuracy,TP,TN,FP,FN,precision,recall,auc,f1_score
0,0,0.293435,0.980985,59.0,999678.0,14540.0,4839.0,0.004041,0.012046,0.513576,0.006052
1,1,0.035892,0.995060,20.0,1014062.0,156.0,4878.0,0.113636,0.004083,0.619871,0.007883
2,2,0.024394,0.995214,53.0,1014185.0,33.0,4845.0,0.616279,0.010821,0.808517,0.021268
3,3,0.022022,0.995267,157.0,1014136.0,82.0,4741.0,0.656904,0.032054,0.843077,0.061125
4,4,0.020792,0.995345,305.0,1014067.0,151.0,4593.0,0.668860,0.062270,0.853154,0.113934
5,5,0.020330,0.995359,414.0,1013972.0,246.0,4484.0,0.627273,0.084524,0.855154,0.148974
6,6,0.019816,0.995417,552.0,1013893.0,325.0,4346.0,0.629418,0.112699,0.856324,0.191169
7,7,0.019586,0.995443,641.0,1013831.0,387.0,4257.0,0.623541,0.130870,0.857391,0.216335
8,8,0.019170,0.995517,798.0,1013749.0,469.0,4100.0,0.629834,0.162924,0.860843,0.258881
9,9,0.018486,0.995659,1011.0,1013681.0,537.0,3887.0,0.653101,0.206411,0.864894,0.313683


In [8]:
val_df = pd.read_csv(f'{path}training_runs/validation_{model.name}.csv')
val_df['f1_score'] = 2 * (val_df['precision'] * val_df['recall']) / (val_df['precision'] + val_df['recall'])
val_df

,epoch,loss,binary_accuracy,TP,TN,FP,FN,precision,recall,auc,f1_score
0,0,0.049498,0.993651,0.0,61348.0,0.0,392.0,0.000000,0.000,0.525060,NaN
1,1,0.033804,0.993651,0.0,61348.0,0.0,392.0,0.000000,0.000,0.805212,NaN
2,2,0.030755,0.993651,0.0,61348.0,0.0,392.0,0.000000,0.000,0.826178,NaN
3,3,0.031669,0.992857,0.0,61299.0,49.0,392.0,0.000000,0.000,0.799021,NaN
4,4,0.030842,0.993651,0.0,61348.0,0.0,392.0,0.000000,0.000,0.824780,NaN
5,5,0.029460,0.993651,49.0,61299.0,49.0,343.0,0.500000,0.125,0.833866,0.200000
6,6,0.031402,0.993651,49.0,61299.0,49.0,343.0,0.500000,0.125,0.841104,0.200000
7,7,0.030957,0.992857,49.0,61250.0,98.0,343.0,0.333333,0.125,0.840555,0.181818
8,8,0.031321,0.993651,49.0,61299.0,49.0,343.0,0.500000,0.125,0.843850,0.200000
9,9,0.030883,0.992857,0.0,61299.0,49.0,392.0,0.000000,0.000,0.845298,NaN


The epoch with the best performance on validation is **15**